In [0]:
import random 
random.randrange(0,3)

1

In [0]:
import random
import numpy as np
import gym

env = gym.make('MountainCar-v0')
env = env.unwrapped

score_requirement = -198

# first off populate the database

episodes = 10000
goal_step = 200
accepted_scores = []
training_data = []


for episode in range(episodes):
  game_memory = []
  score = 0
  previous_state = []
  for step in range(goal_step):
    action = random.randrange(0, 3)
    new_state, reward, done, info = env.step(action)
    
    if len(previous_state) > 0:
      game_memory.append([previous_state, action])
      
    previous_state = new_state
    
    if new_state[0] > -0.2:
      reward = 1
      
    score += reward
    if done:
      break

    if score >= score_requirement:
      
      accepted_scores.append(score)
      for data in game_memory:
        if data[1] == 1:
          output = [0, 1, 0]
        elif data[1] == 0:
          output = [1, 0, 0]
        elif data[1] == 2:
          output = [0, 0, 1]
        training_data.append([data[0], output])

In [2]:
training_data

[[array([-5.13379663e-01, -7.72101406e-05]), [0, 1, 0]],
 [array([-5.13379663e-01, -7.72101406e-05]), [0, 1, 0]],
 [array([-5.13533505e-01, -1.53841479e-04]), [0, 1, 0]],
 [array([-5.13379663e-01, -7.72101406e-05]), [0, 1, 0]],
 [array([-5.13533505e-01, -1.53841479e-04]), [0, 1, 0]],
 [array([-5.13762824e-01, -2.29319541e-04]), [1, 0, 0]],
 [array([-5.13379663e-01, -7.72101406e-05]), [0, 1, 0]],
 [array([-5.13533505e-01, -1.53841479e-04]), [0, 1, 0]],
 [array([-5.13762824e-01, -2.29319541e-04]), [1, 0, 0]],
 [array([-0.5150659 , -0.00130308]), [0, 0, 1]],
 [array([-5.13379663e-01, -7.72101406e-05]), [0, 1, 0]],
 [array([-5.13533505e-01, -1.53841479e-04]), [0, 1, 0]],
 [array([-5.13762824e-01, -2.29319541e-04]), [1, 0, 0]],
 [array([-0.5150659 , -0.00130308]), [0, 0, 1]],
 [array([-5.15432971e-01, -3.67068031e-04]), [0, 1, 0]],
 [array([-5.13379663e-01, -7.72101406e-05]), [0, 1, 0]],
 [array([-5.13533505e-01, -1.53841479e-04]), [0, 1, 0]],
 [array([-5.13762824e-01, -2.29319541e-04]), [1

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))

In [10]:
input_size = len(X[0])
output_size = len(y[0])
model = Sequential()
model.add(Dense(128, input_dim = input_size, activation = 'relu'))
model.add(Dense(52, activation = 'relu'))
model.add(Dense(output_dim = output_size, activation = 'softmax'))
model.summary()
model.compile(loss = 'mse', optimizer = Adam())

print(input_size)
print(output_size)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               384       
_________________________________________________________________
dense_11 (Dense)             (None, 52)                6708      
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 159       
Total params: 7,251
Trainable params: 7,251
Non-trainable params: 0
_________________________________________________________________
2
3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=3)`
  


In [11]:
model.fit(X, y, epochs = 5)

W0904 05:54:47.011597 139829175252864 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0904 05:54:47.106229 139829175252864 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/5
2178712/2178712 [==============================] - 94s 43us/step - loss: 0.2221
Epoch 2/5
2178712/2178712 [==============================] - 82s 38us/step - loss: 0.2219
Epoch 3/5
2178712/2178712 [==============================] - 86s 39us/step - loss: 0.2216
Epoch 4/5
2178712/2178712 [==============================] - 85s 39us/step - loss: 0.2215
Epoch 5/5
2178712/2178712 [==============================] - 85s 39us/step - loss: 0.2214


In [0]:
model.save('/content/MountainCar.hdf5')

In [14]:
!pip install h5py

import h5py
  

In [19]:
model_episodes = 100
model_scores = []
model_actions = []
game_memory = []

for model_episode in range(model_episodes):
  score = 0
  game_steps = 200
#   game_memory = []
  prev_state = []
  for step in range(game_steps):
    if len(prev_state) == 0:
      action = random.randrange(0, 2)
    else:
      with h5py.File('/content/MountainCar.hdf5') as f:
        action = np.argmax(f.predict(prev_state.reshape(-1, len(prev_state)))[0])
    
    new_state, reward, done, info = env.step(action)
    prev_state = new_state
    score += reward
    model_actions.append(action)
    model_scores.append(reward)
    game_memory.append([prev_state, action])
    if done:
      print('-----------------------done--------------------')
      break
      
    

print(score)
print(game_memory)

-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done--------------------
-----------------------done-------------